In [8]:
import os
import dspy

In [9]:
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')

In [12]:
lm = dspy.LM('openai/gpt-4o-mini', api_key=openai_api_key)
dspy.configure(lm=lm)

In [22]:
prompt_poesia = """
## FUNÇÃO
Você é um assistente de escrita criativa com foco em poesia.

## TAREFA
Sua tarefa é gerar associações de palavras que gerem novas ideias para a pessoa que está escrevendo.

Ao receber uma palavra, responda com uma série de palavras e expressões relacionadas a ela. Organize essas palavras em categorias ou temas. Inclua quantas outras achar necessário, da forma que achar melhor.
"""

In [25]:
resposta = lm(messages=[
    {"role": "system", "content": prompt_poesia},
    {"role": "user", "content": "Água"}]
)

In [27]:
formatted_resposta = "\n\n".join(resposta)
print(formatted_resposta)

### Elementos Naturais
- Rio
- Oceano
- Chuva
- Lago
- Fonte
- Gota
- Nuvem

### Emoções e Sensações
- Pureza
- Tranquilidade
- Fluidez
- Refresco
- Renovação
- Serenidade

### Ciclos da Vida
- Crescimento
- Fertilidade
- Ciclo da água
- Vida
- Sustento
- Transformação

### Metáforas e Simbolismos
- Espelho da alma
- Correnteza do tempo
- Fluxo das emoções
- Profundidade
- Reflexão
- Limpeza

### Atividades e Usos
- Natação
- Banho
- Hidratante
- Irrigação
- Navegação
- Pesca

### Cultura e Mitos
- Deuses da água
- Lendas de sereias
- Rituais de purificação
- Contos de inundação
- Mitos de criação

### Palavras Relacionadas
- Hidratação
- Transparência
- Movimento
- Ondas
- Maré
- Salinidade

Essas associações podem inspirar diferentes temas e imagens em sua poesia sobre a água!
